In [14]:
import praw
import re
import random
import bs4, markdown
from tqdm import tqdm

In [7]:
credential_file = "credentials.key"
target_subreddit = "INTP"
try:
    with open(credential_file, 'r') as f:
        creds = f.read().split('\n')
    personal = creds[0]
    secret = creds[1]
    username = creds[2]
    password = creds[3]
except IOError as e:
    print("You didn't create a credential file! Please see sample_credentials.key")
    print("Then go to http://www.storybench.org/how-to-scrape-reddit-with-python/")
    print("And register a new app named fastai_reddit in your reddit account.")
    print("And insert the values into sample_credentials.key and save it as {}.".format(credential_file))
    raise(e)

In [5]:
def noquotes(text):
    """
This function first stated out as a way to remove markdown quotes from raw reddit markdown text but now it's more of a
general purpose text parser, but the name hasn't changed.
    """
    #https://stackoverflow.com/questions/761824/python-how-to-convert-markdown-formatted-text-to-text
    t1 = re.sub(">.+?(\n|$)","",text).replace("\\n","").replace("\\","")
    html = markdown.markdown(t1)
    t2 = ''.join(bs4.BeautifulSoup(html, 'lxml').findAll(text=True))
    
    return t2

In [8]:
reddit = praw.Reddit(client_id=personal, client_secret=secret, user_agent='fastai_reddit', username=username, \
                     password=password)
subreddit = reddit.subreddit(target_subreddit)

In [9]:
test = list(subreddit.top(limit=10))[0]
print(test.title)

It just can't be that simple


In [10]:
posts = list(subreddit.top(limit=10))

In [12]:
def get_comment_text(comment):
    comments = []
    if isinstance(comment, praw.models.MoreComments):
        newcomments = comment.comments()
        for n in newcomments:
            comments += get_comment_text(n)
    else:
        comments.append(noquotes(comment.body))
    return comments
            

In [15]:
alltext = []
for p in tqdm(posts):
    tmp = noquotes(p.selftext)
    if len(tmp) > 0:
        alltext.append(tmp)
    for c in p.comments.list():
        alltext += get_comment_text(c)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.77s/it]


In [16]:
alltext

["Okay, but seriously, that guy's response was epic.",
 'What if his parents named him "4" and his sister "2" and he recently changed his name to "44"? It would make his question a non sequitur and indeterminable without more precise information.',
 '(Spoken in Bill Clinton voice)\nDear Sam Lipoff,\nI feel your pain. Having a brain that works like this is exhausting.',
 '[deleted]',
 'When this guy does it, he\'s "smart"\nWhen I do it, I\'m "fun at parties"',
 'Great answer to what was an incredibly stupid question.',
 'I like how it starts with "this is a tough one" and in the end you realise  he\'s half serious actually.',
 'Found the QA engineer.',
 'Leap year?',
 "Look at this joker. Doesn't even give a complete answer. What about Benjamin Button disease where you age backwards, or that one from Jack where you age 4 times as quick?",
 "Terrible pun, I know, but she's 44 too",
 'Smart. I like this guy.',
 "4 divided by 2 is 2, so you just divide 44 by 2 so obviously she's 22. Simple

In [17]:
len(alltext)

1398